In [1]:
import os
import re
import sys
import ssl
import json
import nltk
import time
import math
import string
import pickle
import argparse

import torch
import numpy as np
import pandas as pd
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter, OrderedDict
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence

In [2]:
# trn = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_train.pkl')
# val = pd.read_pickle('jigsaw_toxic/20pct_model_35toxic_val.pkl')

In [2]:
import jigsaw_toxic.exec_lstm as exl

Using CUDA? True cuda:0


In [3]:
torch.cuda.is_available()

True

In [5]:
# X_train = trn.drop('label', axis=1)
# y_train = trn['label']
# test_sample = val.sample( n=math.ceil(len(X_train)*0.2), random_state=1008 )
# X_test = test_sample.drop('label', axis=1)
# y_test = test_sample['label'] 

In [6]:
# lstm_model = exl.LSTMModel(X_train[:500], y_train[:500], X_test[:100], y_test[:100], 
#                            hidden_dim=50, num_layers=1, embed_dim=50, batch_size=1,
#                            dropout=0, num_classes=2)

In [11]:
# if exl.USE_CUDA:
#     print(exl.USE_CUDA)
#     lstm_model = lstm_model.cuda()

In [8]:
# lstm_model.train()

LSTMModel(
  (loss_fcn): NLLLoss()
  (embedding): Embedding(7500, 50)
  (lstm): LSTM(50, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=2, bias=True)
)

In [9]:
# NUM_EPOCHS = 6
# hist_lstm = np.zeros(NUM_EPOCHS)
# model_name="toy_dataset"

In [10]:
# results, model_state_dict = lstm_model.run_model(y_train[:500], X_test[:100], y_test[:100], NUM_EPOCHS, 
#                                                  hist_lstm, text_col='cleaned_no_stem', savestate=model_name) 

Epochs: 6; Train Size: 500; Test Size: 100)
Starting epoch 0...
Epoch 0, Negative Log Linear Loss: 0.38605058193206787

Starting Evaluation


/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 0 F1 Score: 0.0
Starting epoch 1...

Starting Evaluation
Epoch 1 F1 Score: 0.0
Starting epoch 2...
Epoch 2, Negative Log Linear Loss: 0.3697086274623871

Starting Evaluation
Epoch 2 F1 Score: 0.0
Starting epoch 3...

Starting Evaluation
Epoch 3 F1 Score: 0.0
Starting epoch 4...
Epoch 4, Negative Log Linear Loss: 0.36790764331817627

Starting Evaluation
Epoch 4 F1 Score: 0.0
Starting epoch 5...

Starting Evaluation
Epoch 5 F1 Score: 0.0
Highest F1 Score: 0.0, Epoch: 0


In [11]:
# exl.get_metrics(results, should_print=True, detailed = False, 
#                 label_col="y_true", score_col="predicted_score")

Group Size: 100
Overall Accuracy: 0.89
Overall Precision: 0.0
Overall Recall: 0.0
Overall F1 Score: 0.0
ROC_AUC: 0.5

Group Size: 11
Target Accuracy: 0.0
Target Precision: 0.0
Target Recall: 0.0
Target F1 Score: 0.0

Group Size: 89
Non-Target Accuracy: 1.0
Non-Target Precision: 1.0
Non-Target Recall: 1.0
Non-Target F1 Score: 1.0



/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhinkson/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'Overall': {'Accuracy': 0.89,
  'Precision': 0.0,
  'Recall': 0.0,
  'F1': 0.0,
  'ROC_AUC': 0.5},
 'Target': {'Accuracy': 0.0, 'Precision': 0.0, 'Recall': 0.0, 'F1': 0.0},
 'Non-Target': {'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0}}

### Creating Batch Loader to Enable Larger Dataset Training

In [11]:
train_full = pd.read_csv("jigsaw_toxic/train.csv")

In [12]:
train_full['label'] = train_full.target.apply(lambda x: 1 if x > 0.5 else 0)

In [252]:
train_full['label_multiclass'] = train_full.target.apply(
    lambda x: "very toxic" if x > 0.6 else "toxic" if 0.4 < x <= 0.6 else "not_toxic")


In [253]:
test_ratio = 0.2

In [254]:
train_sample, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

Retrieving training samples at proportion 0.2


In [255]:
print(train_sample.shape, val_set.shape, test_set.shape)

(288740, 47) (181156, 47) (180019, 47)


In [256]:
val_samples = val_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [257]:
val_samples.shape

(57748, 47)

In [146]:
#  Attribution: https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
        examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
            string is a field name, and the Field is the associated field.
        filter_pred (callable or None): use only exanples for which
            filter_pred(example) is true, or use all examples if None.
            Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

                
class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
#             print(key)
#             print(field)
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [19]:
def tokenizer(text, stop_ws=exl.stops, stemmer=None, str_output=False):

    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]

    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]

    if stemmer:
        t = [stemmer.stem(w) for w in t]

    if str_output:
        return ' '.join(t)
    else:
        return t

In [141]:
from torchtext.data import Field, BucketIterator, TabularDataset

TEXT = Field(tokenize=tokenizer, lower=True, init_token = "<sos>", 
             eos_token = "<eos>", batch_first=True)
LABEL = data.Field(sequential=False, unk_token=None, dtype=torch.float, is_target=True)
# LABEL = data.Field(sequential=False, unk_token=None, dtype=torch.float)

In [142]:
train_ds = DataFrameDataset(train_sample, fields = { 'label' : LABEL, 'comment_text' : TEXT })
valid_ds = DataFrameDataset(val_samples, fields = { 'label' : LABEL, 'comment_text' : TEXT })


In [ ]:
train_ds2 = DataFrameDataset(train_sample, fields = { 'label_multiclass' : LABEL, 'comment_text' : TEXT })
valid_ds2 = DataFrameDataset(val_samples, fields = { 'label_multiclass' : LABEL, 'comment_text' : TEXT })


In [169]:
TEXT.build_vocab(train_ds, max_size=25000)
LABEL.build_vocab(train_ds)

In [170]:
print(TEXT.vocab.stoi['leave'])

434


In [171]:
print(TEXT.vocab.itos[434])

leave


In [172]:
# Attribution: Adapted from https://towardsdatascience.com/how-to-use-torchtext-for-neural-machine-translation-plus-hack-to-make-it-5x-faster-77f3884d95
import torch
from torchtext import data
import numpy as np

global max_text_in_batch

def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_text_in_batch
    if count == 1:
        max_text_in_batch = 0
    max_text_in_batch = max(max_text_in_batch, len(new.comment_text) + 2)
    text_elements = count * max_text_in_batch
    return text_elements


class EfficientIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

In [179]:
train_ds.fields

{'label': <torchtext.data.field.Field at 0x7f7c855a2a20>,
 'comment_text': <torchtext.data.field.Field at 0x7f7c855a2ac8>}

In [180]:
valid_ds.fields

{'label': <torchtext.data.field.Field at 0x7f7c855a2a20>,
 'comment_text': <torchtext.data.field.Field at 0x7f7c855a2ac8>}

In [173]:
print('len(TEXT.vocab)', len(TEXT.vocab))
# print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

len(TEXT.vocab) 25004


In [190]:
VOCAB_SIZE = len(TEXT.vocab)

In [159]:
from torchtext.vocab import GloVe

In [162]:
# !wget -P ../ "http://nlp.stanford.edu/data/glove.840B.300d.zip"

In [163]:
# !unzip "../glove.840B.300d.zip"

In [181]:
train_iter = BucketIterator(train_ds, batch_size=20, \
sort_key=lambda x: len(x.comment_text), shuffle=True)

In [182]:
batch = next(iter(train_iter))
print(batch.comment_text)

tensor([[   2,   23, 3085,  ...,    1,    1,    1],
        [   2,   68,  617,  ...,    1,    1,    1],
        [   2,  428, 1508,  ...,    1,    1,    1],
        ...,
        [   2,  404, 1091,  ...,    1,    1,    1],
        [   2,   36,  117,  ...,    1,    1,    1],
        [   2,    4,   17,  ...,  894,    8,    3]])


In [183]:
train_iter = EfficientIterator(
    train_ds, batch_size=1300, 
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
    train=True, shuffle=True)


In [227]:
train_iter = BucketIterator(train_ds, batch_size=1300, \
sort_key=lambda x: len(x.comment_text), shuffle=True)

In [196]:
val_iter = EfficientIterator(
    valid_ds, batch_size=1300, 
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
    train=True, shuffle=True)


In [230]:
batch = next(iter(train_iter))
print(batch.comment_text)
print(batch.label)

tensor([[   2,   36,  565,  ...,    1,    1,    1],
        [   2, 5079,  705,  ...,    1,    1,    1],
        [   2, 4821, 1623,  ...,    1,    1,    1],
        ...,
        [   2,  135, 1005,  ...,    1,    1,    1],
        [   2,    6, 1633,  ...,    1,    1,    1],
        [   2,  782,  187,  ...,    1,    1,    1]])
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [197]:
batch = next(iter(val_iter))
print(batch.comment_text)

tensor([[    2,    25,     9,  ...,    37,  1324,     3],
        [    2,   436,  2040,  ...,   793,  7787,     3],
        [    2,  2629,  1109,  ...,     0,  3532,     3],
        ...,
        [    2,     4,    12,  ...,   312,    70,     3],
        [    2,    11,   255,  ...,    24,   159,     3],
        [    2,    80, 19986,  ...,  5688,   141,     3]], device='cuda:0')


In [239]:
class LSTMModel(nn.Module):
    def __init__(self, hidden_dim, batch_size=1,
                 embed_dim=6, weight_decay=0, optimizer_fcn='Adam',
                 learning_rate=1e-3, num_layers=2, dropout=0.05, num_classes=2):
        super(LSTMModel, self).__init__()
        nn.Module.__init__(self)
#         TextData.__init__(self, X_data)
        self.vocab_size = VOCAB_SIZE
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.output_dim = num_classes
        self.loss_fcn = nn.NLLLoss()
        self.weight_decay = weight_decay
        self.learning_rate = learning_rate
        self.highest_f1 = -math.inf

        # Layer 1: Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_dim)

        # Layer 2: LSTM Layer
        self.lstm = nn.LSTM(input_size = self.embed_dim, hidden_size = self.hidden_dim,
                            num_layers = self.num_layers, dropout = self.dropout, 
                            batch_first=True)

        # Layer 3 (Output Layer): Linear
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)

        # define optimizer
        if optimizer_fcn == 'Adam':
            self.optimizer = optim.Adam(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'RMSprop':
            self.optimizer = optim.RMSprop(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'SDG':
            self.optimizer = optim.SGD(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)

    def forward(self, input_seq):

        embed_out = self.embedding(input_seq)

        lstm_out, (hn, cn) = self.lstm(embed_out)

        out = F.log_softmax(self.linear(hn), dim=self.output_dim)

        return out


In [240]:
NUM_EPOCHS = 6
hist_lstm = np.zeros(NUM_EPOCHS)
model_name="toy_dataset"

In [241]:
lstm_model = LSTMModel(hidden_dim=50, num_layers=1, embed_dim=50, 
                       batch_size=train_iter.batch_size,
                       dropout=0, num_classes=2)

In [242]:
torch.cuda.is_available()

True

In [243]:
if torch.cuda.is_available():
    lstm_model = lstm_model.cuda()

In [251]:
results = []
last_mean_error = -math.inf

for epoch in range(NUM_EPOCHS):
    running_loss = 0
    for x, y in train_iter:
#         print(x)
#         print(x.size())
        print(y)
#         print(y.size()) 
        if torch.cuda.is_available():
            x = x.cuda() 
            y = y.cuda()
            
        lstm_model.optimizer.zero_grad()
        
        embed_out = lstm_model.embedding(x)

        print("embed", embed_out.size())
        lstm_out, (hn, cn) = lstm_model.lstm(embed_out)
        print("hn", hn.size())
        print("lstm_out", lstm_out.size())
        print("cn", cn.size())

#         out = F.log_softmax(lstm_model.linear(hn), dim=lstm_model.output_dim)
        out = F.log_softmax(lstm_model.linear(lstm_out), dim=lstm_model.output_dim)
        print("original", out)
        print(out.size())
        print("view -1", out.view(-1))
        print(out.view(-1).size())
        print("unsqueeze 1", out.unsqueeze(1))
        print(out.unsqueeze(1).size())
        print("unsqueeze 0", out.unsqueeze(0))
        print(out.unsqueeze(0).size())
        print("labels", y)
        print(y.size())
        loss = lstm_model.loss_fcn(out, y)
        loss.backward()
        
        running_loss += loss.item()
        
        self.optimizer.step()
        
        thresholds= {
                5000: 1000,
                1000: 100,
                100: 10,
                20: 5,
                10: 3
            }
        
        print_at = 2
        if num_epochs in  thresholds.keys():
            print_at = thresholds[num_epochs]
        # print results for every few epochs
        if epoch % print_at == 0:
            print(f"Epoch {epoch}, Negative Log Linear Loss: {running_loss}")
            hist_lstm[epoch] = loss.item()

            with torch.no_grad():
                if np.mean(np.abs(loss.item())) < last_mean_error:
                    print(f"Delta after {epoch} iterations: {np.mean(np.abs(loss.item()))}")
                    last_mean_error = np.mean(np.abs(loss.item()))
                else:
                    if last_mean_error > -math.inf:
                        print(f"Break: {np.mean(np.abs(loss.item()))} > {last_mean_error}")
                        break
        print()
        print("Starting Evaluation")                    
#         model_f1, results_df = self.evaluate_classifier(test_X, test_y, text_col=text_col)
        
        with torch.no_grad():
            for text, labels in val_iter:
                correct = 0
                total = 0
                if torch.cuda.is_available():
                    text = text.cuda() 
                    labels = labels.cuda()

                lstm_model.optimizer.zero_grad()

                embed_out = lstm_model.embedding(text)

                lstm_out, (hn, cn) = lstm_model.lstm(embed_out)

                preds = F.log_softmax(lstm_model.linear(hn), dim=lstm_model.output_dim)
                print(preds)
                print(preds.size())
                print(labels)
                print(labels.size)
                indices = torch.argmax(preds, dim=0)
                print(indices)

                _, predicted = torch.max(preds.data, 1)
                print(predicted)
                correct += float((predicted == labels).sum().item())


tensor([0., 0., 0.,  ..., 0., 1., 0.])
embed torch.Size([1300, 118, 50])
hn torch.Size([1, 1300, 50])
lstm_out torch.Size([1300, 118, 50])
cn torch.Size([1, 1300, 50])
original tensor([[[-0.7926, -0.6027],
         [-0.8277, -0.5745],
         [-0.7621, -0.6286],
         ...,
         [-0.7104, -0.6762],
         [-0.7104, -0.6762],
         [-0.7104, -0.6762]],

        [[-0.7926, -0.6027],
         [-0.7133, -0.6734],
         [-0.6564, -0.7313],
         ...,
         [-0.7104, -0.6762],
         [-0.7104, -0.6762],
         [-0.7104, -0.6762]],

        [[-0.7926, -0.6027],
         [-0.7413, -0.6472],
         [-0.7550, -0.6349],
         ...,
         [-0.7104, -0.6762],
         [-0.7104, -0.6762],
         [-0.7104, -0.6762]],

        ...,

        [[-0.7926, -0.6027],
         [-0.7989, -0.5975],
         [-0.7720, -0.6201],
         ...,
         [-0.7104, -0.6762],
         [-0.7104, -0.6762],
         [-0.7104, -0.6762]],

        [[-0.7926, -0.6027],
         [-0.7735, -

ValueError: Expected target size (1300, 2), got torch.Size([1300])